<a href="https://colab.research.google.com/github/danielyangdev/Heart-Disease-Detector/blob/main/Heart_Disease_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, ReLU

In [ ]:
myData = pd.read_csv('heart.csv')
myData = (myData - np.min(myData))/(np.max(myData) - np.min(myData))

x = myData.drop('target', axis=1)
y = myData['target']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.2, random_state=2)

In [5]:
model = Sequential()
model.add(Dense(100, input_shape=(13,),activation="relu"))
model.add(Dense(units=64, activation='relu')),
model.add(Dense(units=128, activation='relu')),
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='categorical_hinge', metrics=['accuracy'])

In [6]:
hist=model.fit(X_train, Y_train,validation_data=(X_test, Y_test), epochs=10)
y_predicted = (model.predict(X_test) >= 0.5)

conf_mat = confusion_matrix(Y_test, y_predicted)
total = sum(sum(conf_mat))
accuracy = (conf_mat[0, 0] + conf_mat[1, 1])/total
print('accuracy : ', accuracy)
#Accuracy = 88.7%

Epoch 1/10
26/26 [==============================] - 1s 12ms/step - loss: 0.9205 - accuracy: 0.7207 - val_loss: 0.8533 - val_accuracy: 0.7902
Epoch 2/10
26/26 [==============================] - 0s 4ms/step - loss: 0.7447 - accuracy: 0.8037 - val_loss: 0.7218 - val_accuracy: 0.8098
Epoch 3/10
26/26 [==============================] - 0s 4ms/step - loss: 0.6734 - accuracy: 0.8317 - val_loss: 0.7159 - val_accuracy: 0.8000
Epoch 4/10
26/26 [==============================] - 0s 3ms/step - loss: 0.6587 - accuracy: 0.8439 - val_loss: 0.6833 - val_accuracy: 0.8537
Epoch 5/10
26/26 [==============================] - 0s 3ms/step - loss: 0.6431 - accuracy: 0.8512 - val_loss: 0.6792 - val_accuracy: 0.8439
Epoch 6/10
26/26 [==============================] - 0s 3ms/step - loss: 0.6348 - accuracy: 0.8610 - val_loss: 0.6655 - val_accuracy: 0.8537
Epoch 7/10
26/26 [==============================] - 0s 4ms/step - loss: 0.6231 - accuracy: 0.8720 - val_loss: 0.6617 - val_accuracy: 0.8585
Epoch 8/10
26/26 [=